In [ ]:

storageAccount="cookbookstoragegen2"
mountpoint = "/mnt/Gen2"
storageEndPoint ="abfss://rawdata@{}.dfs.core.windows.net/".format(storageAccount)
print ('Mount Point ='+mountpoint)

#ClientId, TenantId and Secret is for the Application(ADLSGen2App) was have created as part of this recipe
clientID ="asdasda-sdas-asdasdasdqsda"
tenantID ="asdasd6f1-41aasdasasdsd-12312"
clientSecret =""
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": clientID,
"fs.azure.account.oauth2.client.secret": clientSecret,
"fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}

dbutils.fs.mount(
source = storageEndPoint,
mount_point = mountpoint,
extra_configs = configs)

Mount Point =/mnt/Gen2
Out[3]: True

In [ ]:
# dbutils.fs.unmount("/mnt/Gen2/")

/mnt/Gen2/ has been unmounted.
Out[2]: True

In [ ]:
display(dbutils.fs.ls("/mnt/Gen2/LogFiles"))

path,name,size
dbfs:/mnt/Gen2/LogFiles/Log1.json,Log1.json,211
dbfs:/mnt/Gen2/LogFiles/Log2.json,Log2.json,209
dbfs:/mnt/Gen2/LogFiles/Log3.json,Log3.json,209
dbfs:/mnt/Gen2/LogFiles/Log4 - Copy.json,Log4 - Copy.json,209
dbfs:/mnt/Gen2/LogFiles/Log4.json,Log4.json,209


In [ ]:
%fs head dbfs:/mnt/Gen2/LogFiles/Log3.json

{
 "source": "Application 1",
 "data": "Log data example 1",
 "IP": "10.22.135.7",
 "log_level": "DEBUG",
 "log_type": "Error",
 "log_app": "app",
 "log_timestamp": "2020-08-25T17:45:13+07:00"
}

In [ ]:
from pyspark.sql.types import *

logFilePath = "/mnt/Gen2/LogFiles/"

logSchema = StructType([ StructField("source", StringType(), True), StructField("data", StringType(), True), StructField("IP", StringType(), True), StructField("log_level", StringType(), True),StructField("log_type", StringType(), True),StructField("log_app", StringType(), True), StructField("log_timestamp", TimestampType(), True) ])


LogDF = (
  spark
    .read
    .schema(logSchema)
    .option("multiline","true")
    .json(logFilePath)
)

display(LogDF)



source,data,IP,log_level,log_type,log_app,log_timestamp
Application 1,Log data example 1,10.22.155.7,DEBUG,Warning,app,2020-08-25T07:45:13.000+0000
Application 1,Log data example 1,10.22.135.7,DEBUG,Error,app,2020-08-25T08:45:13.000+0000
Application 1,Log data example 1,10.22.135.7,DEBUG,Error,app,2020-08-25T10:45:13.000+0000


In [ ]:
from pyspark.sql.functions import *      

logCountDF = (
  LogDF
    .groupBy(col("log_type"))       
    .count()
)
display(logCountDF)

log_type,count
Error,2
Warning,1


In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

logFilePath = "/mnt/Gen2/LogFiles/"

logSchema = StructType([ StructField("source", StringType(), True), StructField("data", StringType(), True), StructField("IP", StringType(), True), StructField("log_level", StringType(), True),StructField("log_type", StringType(), True),StructField("log_app", StringType(), True), StructField("log_timestamp", TimestampType(), True) ])


streamingLogDF = (
  spark
    .readStream
    .schema(logSchema)       
    .option("multiline","true")
    .option("maxFilesPerTrigger", 1)
    .json(logFilePath)
)


streamingDataCountDF = (
  streamingLogDF
    .groupBy(col("log_type"))       
    .count()
)
display(streamingDataCountDF)

log_type,count
Error,4
Warning,1


In [ ]:
report = (
  streamingDataCountDF
    .writeStream
    .format("memory")        
    .queryName("counts")    
    .outputMode("complete")  
    .start()
)

In [ ]:
%sql
select * from counts

log_type,count
Error,3
Warning,1
